<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train5_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd

In [220]:

train5 = pd.read_csv(r'/content/train_data5.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [221]:

train5['Target'] = train5['Target'].replace(['Others'],'Invalid')


In [222]:
train5['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [223]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [224]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train5_Xtfidf = tfidf_vect.fit_transform(train5['Sentence'])


In [225]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [226]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [227]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [228]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [229]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [230]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [231]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train5_Xtfidf,train5['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6836283185840708
                  precision    recall  f1-score   support

        Analysis       0.53      0.41      0.46       126
      Conclusion       0.78      0.56      0.65        45
           Facts       0.74      0.92      0.82       483
         Invalid       0.52      0.42      0.46       133
           Issue       0.55      0.44      0.49        48
Rule/Law/Holding       0.69      0.26      0.38        69

        accuracy                           0.68       904
       macro avg       0.64      0.50      0.54       904
    weighted avg       0.67      0.68      0.66       904



In [234]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [235]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]
[0.9919723242439552, 0.9932611784994131, 0.9946162588862182, 0.9999549951199664, 0.9989696983834331, 0.9968960343628208, 0.9949886755589121, 0.999129774518202, 0.9972246236812229, 0.9894564128199387, 0.9931725968278492, 0.9993566834550314, 0.997842646734431, 0.9993145962799928, 0.9887585691061426, 0.9997214556197167, 0.9874028301259222, 0.9823524186743089, 0.9843747378721157, 0.9902902439104883, 0.991972750999075, 0.9973766103128284, 0.9984114114842711, 0.9995406766072056, 0.9997625062437582, 0.9911748979134892, 0.9966689261469902, 0.9920606318889157, 0.9939765341657052, 0.9953320676957692, 0.9959057110023164, 0.9963965902658305, 0.9984915315350469, 0.9872127429662904, 0.99436884138754, 0.9843299479507978, 0.9998183028731985, 0.9886707123546742, 0.9845992259133395, 0.9904041793076581, 0.9834130517467691, 0.99963116208782, 0.9958614838425364, 0.99935844

In [236]:
unlabel_1=unlabel_1.loc[[0, 1, 4, 5, 11, 12, 17, 18, 19, 20, 21, 23, 24, 29, 30, 31, 32, 33, 35, 36, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 60, 62, 65, 68, 70, 76, 79, 84, 87, 89, 94, 95, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
11,1604,police notified deceased found lying mrs moers...,Facts


In [237]:
frame_1 = [train5,unlabel_1]
train5_1 = pd.concat(frame_1)
len(train5_1)

3521

In [238]:
x_train_1 = tfidf_vect.transform(train5_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train5_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6553191489361702
                  precision    recall  f1-score   support

        Analysis       0.48      0.48      0.48        98
      Conclusion       0.79      0.55      0.65        42
           Facts       0.72      0.86      0.79       344
         Invalid       0.52      0.54      0.53       112
           Issue       0.70      0.48      0.57        48
Rule/Law/Holding       0.61      0.23      0.33        61

        accuracy                           0.66       705
       macro avg       0.64      0.52      0.56       705
    weighted avg       0.65      0.66      0.64       705



In [239]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2]
[0.9968397111673811, 0.9990531277463959, 0.9977748574518019, 0.9932433681313056, 0.9961670156809265, 0.9965713996239164, 0.9999376919686089, 0.9921064178386124, 0.9990490259571408, 0.9962770577111605, 0.9847164629609647, 0.9981736998749448, 0.9955806200069387, 0.9986337714649846, 0.9999617367017032, 0.9994050836453934, 0.9935022761857715, 0.9871927126101351, 0.9984659202982525, 0.9867862014034048, 0.9985304078467516, 0.9931443934451925, 0.9836024897300989, 0.9995986899398145, 0.9875326781203377, 0.9987133378255215, 0.9998968410446363, 0.9987560736900846, 0.999542282869454, 0.9970497458090425, 0.9938587843535217, 0.9955048934155557, 0.981272430126756, 0.9985502765579699, 0.9996305703074766, 0.9992934273877603, 0.9811420168163238, 0.9892096212237625, 0.9976098977044767]
[0, 2, 4, 6, 7, 8, 12, 17, 20, 21, 23, 29, 34, 36, 37, 39, 42, 44, 49, 50, 54, 55, 58, 59, 60, 63, 64, 

In [240]:
unlabel_2 = unlabel_2.loc[[0, 2, 4, 6, 7, 8, 12, 17, 20, 21, 23, 29, 34, 36, 37, 39, 42, 44, 49, 50, 54, 55, 58, 59, 60, 63, 64, 69, 70, 71, 75, 83, 84, 85, 88, 91, 92, 95, 99],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
2,2072,appellant testify offer evidence behalf,Facts
4,906,agreed accompany gave pawn ticket money reclai...,Facts
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [241]:
frame_2 = [train5_1,unlabel_2]
train5_2 = pd.concat(frame_2)
len(train5_2)

3560

In [242]:
x_train_2 = tfidf_vect.transform(train5_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train5_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6611226611226612
                  precision    recall  f1-score   support

        Analysis       0.49      0.46      0.47       142
      Conclusion       0.69      0.36      0.47        50
           Facts       0.73      0.90      0.81       486
         Invalid       0.54      0.46      0.49       164
           Issue       0.56      0.33      0.42        60
Rule/Law/Holding       0.66      0.35      0.46        60

        accuracy                           0.66       962
       macro avg       0.61      0.48      0.52       962
    weighted avg       0.65      0.66      0.64       962



In [243]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9967364056184043, 0.9982853169279176, 0.9950300927133755, 0.9865944565970733, 0.9996607293266405, 0.9997060452141487, 0.9994144063019306, 0.9996856836922713, 0.9911778396455394, 0.9998611863174944, 0.9980269303901411, 0.9983154464897935, 0.9992764418073713, 0.9922378369013438, 0.9993221457838893, 0.9983917385770363, 0.992535871467593, 0.9885214121959118, 0.9971417404757379, 0.9937901561462453, 0.9897046196221664, 0.9985829691328534, 0.9996566557607507, 0.9939676020826387, 0.9999038674556177, 0.9951515409455569, 0.9961440004183247, 0.9995417588924372, 0.981155729217181, 0.9992705658101534, 0.9890787405114889, 0.9990546224073749, 0.9886197012796677, 0.9918744801586098, 0.99959318811807, 0.9842838986420892]
[0, 2, 3, 4, 7, 8, 12, 20, 26, 27, 29, 30, 35, 36, 43, 46, 47, 48, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 72, 73, 75, 78, 79, 89, 96]
36
36
36


In [244]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 7, 8, 12, 20, 26, 27, 29, 30, 35, 36, 43, 46, 47, 48, 52, 54, 56, 57, 63, 64, 65, 66, 67, 68, 70, 72, 73, 75, 78, 79, 89, 96],:]
frame_3 = [train5_2,unlabel_3]
train5_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train5_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train5_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6722222222222223
                  precision    recall  f1-score   support

        Analysis       0.44      0.45      0.45        95
      Conclusion       0.71      0.51      0.59        47
           Facts       0.77      0.89      0.83       381
         Invalid       0.49      0.40      0.44       112
           Issue       0.67      0.54      0.60        37
Rule/Law/Holding       0.48      0.25      0.33        48

        accuracy                           0.67       720
       macro avg       0.59      0.51      0.54       720
    weighted avg       0.65      0.67      0.66       720



In [245]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2]
[0.9852235296701826, 0.9996538085900878, 0.9988763055887485, 0.9918373748330283, 0.9962390412531285, 0.9997662773838295, 0.9855717618344005, 0.9967227727547714, 0.9843165067429787, 0.9813949694318591, 0.9997898399780722, 0.999550028894036, 0.9984560686305812, 0.9973620753071435, 0.9991630862722676, 0.9996901625730223, 0.9986164966600332, 0.9988358497864088, 0.9996882931157985, 0.998301493902163, 0.9982963182101319, 0.9867747267751719, 0.9993799480272889, 0.9957143734865571, 0.9885100422796049, 0.9919089717778325, 0.9970902102917941, 0.9843769431514212, 0.9975153941794069, 0.9934521984222416, 0.9927221053236926, 0.9930131573879254, 0.9887901493976119, 0.9990581088785053, 0.9918133638881519, 0.9996381313357705, 0.9906913596530824, 0.9908716989780363, 0.9999471118409673, 0.998988292828446, 0.9870776256490255, 0.9878930503202832, 0.9804011245733852, 0.9873249

In [246]:
unlabel_4=unlabel_4.loc[[0, 2, 4, 6, 7, 10, 11, 15, 16, 17, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 40, 42, 48, 49, 50, 54, 56, 58, 60, 64, 66, 67, 69, 70, 71, 72, 75, 87, 93, 94, 97, 99 ],:]

In [247]:
frame_4 = [train5_3,unlabel_4]
train5_4 = pd.concat(frame_4)
len(train5_4)
x_train_4 = tfidf_vect.transform(train5_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train5_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6622583926754833
                  precision    recall  f1-score   support

        Analysis       0.50      0.36      0.42       154
      Conclusion       0.61      0.50      0.55        46
           Facts       0.73      0.91      0.81       511
         Invalid       0.55      0.42      0.48       156
           Issue       0.52      0.38      0.44        60
Rule/Law/Holding       0.55      0.38      0.45        56

        accuracy                           0.66       983
       macro avg       0.58      0.49      0.52       983
    weighted avg       0.64      0.66      0.64       983



In [248]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 0, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 5, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0]
[0.9991487934263904, 0.9980704107557419, 0.9827256767843772, 0.9919783461442113, 0.9823886508563191, 0.9900818599961193, 0.991880164917326, 0.9877917027721497, 0.9989404311039858, 0.9992366457464025, 0.9929245808870854, 0.9824259150692902, 0.9921370937154786, 0.9994527516486907, 0.9851240706764928, 0.9878564550452539, 0.9925399172106655, 0.9996691275894033, 0.9980933468309506, 0.9813129234363722, 0.9980371604554937, 0.9878802542683551, 0.9973894683019406, 0.9815463614039641, 0.9892218979336328, 0.9882358285796069, 0.9915748619438779, 0.9862683287907174, 0.9999125508643368, 0.9891611995103339, 0.982490149858323, 0.9991209785815441, 0.999640079914625, 0.982349904822398, 0.9917843466283022, 0.9811900885299794, 0.9823599673922256]
[3, 7, 12, 16, 17, 18, 19, 22, 23, 26, 27, 29, 32, 36, 40, 46, 52, 53, 60, 63, 65, 67, 73, 74, 75, 77, 80, 81, 82, 86, 92, 94, 95, 99, 100, 103, 104]
3

In [249]:
unlabel_5 = unlabel_5.loc[[3, 7, 12, 16, 17, 18, 19, 22, 23, 26, 27, 29, 32, 36, 40, 46, 52, 53, 60, 63, 65, 67, 73, 74, 75, 77, 80, 81, 82, 86, 92, 94, 95, 99, 100, 103, 104],:]

In [252]:
frame_5 = [train5_4,unlabel_5]
train5_5 = pd.concat(frame_5)
len(train5_5)
x_train_5 = tfidf_vect.transform(train5_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train5_5['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6878147029204431
                  precision    recall  f1-score   support

        Analysis       0.53      0.46      0.49       147
      Conclusion       0.67      0.51      0.58        47
           Facts       0.76      0.89      0.82       531
         Invalid       0.53      0.45      0.49       147
           Issue       0.60      0.48      0.54        54
Rule/Law/Holding       0.60      0.37      0.46        67

        accuracy                           0.69       993
       macro avg       0.62      0.53      0.56       993
    weighted avg       0.67      0.69      0.67       993



In [253]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.6717850287907869
